# Demonstration of Continuous data event recognition using Semantic Segmentation Models

## Import Dependencies

In [ ]:
import sys
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
from torch import tensor, cuda
import pandas as pd
import requests

### Add utils to sys to import our custom scripts
**Note:** This notebook should be run from the `examples` folder to ensure the correct imports and file paths are used.

In [ ]:
# Go up one level from 'examples' and point to 'utils' folder
utils_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'utils'))
# Add the 'utils' folder to sys.path
sys.path.append(utils_path)
models_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'models'))
# Add the 'models' folder to sys.path
sys.path.append(models_path)
import data_utils
import model_utils
import real_time_utils

### Load the model

Model can be loaded through the model_selector() function. Available architectures are: "UNet", "UNetPlusPlus", "DeepLabV3", "SwinUNet", and PhaseNet. If you wish to load the weights generated by the article, you can set the pretrained argument to True, it automatically downloads the weights from [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.15098817.svg)](https://doi.org/10.5281/zenodo.15098817) (weights are only available for input size N=256). Number of parameters for each model at each input size evaluated in the article is shown below:

In [ ]:
device = "cuda" if cuda.is_available() == True else "cpu"
model = model_utils.model_selector(arch='UNetPlusPlus', N=256, pretrained=True).to(device)

### Load continuous trace

Data can be downloaded from [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.15384923.svg)](https://doi.org/10.5281/zenodo.15384923):

In [ ]:
doi = "10.5281/zenodo.15384923"
record_id = doi.split(".")[-1]
metadata_url = f"https://zenodo.org/api/records/{record_id}"
response = requests.get(metadata_url)
metadata = response.json()
files = metadata["files"]
file_to_download = files[2]  # 0: NVCh_10h_continuous_trace.zip - https://zenodo.org/api/records/15384923/files/NVCh_10h_continuous_trace.zip/content
file_url = file_to_download["links"]["download"]
filename = file_to_download["key"] 
response = requests.get(file_url, stream=True)

with open(filename, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)
print(f"Downloaded {filename}")

print(f"Extracting files...")

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())  # or use a specific path
print(f"Files extracted")

After downloading we load the full seismic trace and the reference dataframe

In [ ]:
continuous_trace = np.load(filename)
reference = pd.read_csv("big_trace_reference_v1.csv")


In [ ]:
reference.value_counts("event_type")

## Sliding window:

We apply our model in a sliding window setup. Stride can be manually selected and is by default set to the lighter value of 7000 (70 seconds). Batch size can also be selected and is by default 16

In [ ]:
batch_size = 16
window_size = 8192
stride = 7000

n_windows = ((continuous_trace.shape[1] - window_size) // stride) + 1
activations = np.zeros([6,continuous_trace.shape[1]])
work_path = f"D:/Camilo/Volcanes_UFRO/RESULTADOS/general_performance/"

In [ ]:
with model_utils.SuppressPrint():
    model.eval()
window_index = 0
for batch in real_time_utils.generate_overlapping_batches(
    continuous_trace, window_size=window_size, stride=stride, batch_size=batch_size,noise_norm=noise_norm
):
    if window_index % 50 == 0:
        print(window_index, "/", n_windows)
    X = data_utils.patch_stacking_X(tensor(batch[:, 1:9, :])).to(device)
    output = model(X)
    output = data_utils.activation_unstacking(output, window_size, N=256, n_classes=6)
    for idx in range(len(output)):
        offset = window_index * stride            
        probabilities = output[idx].detach().cpu().numpy()
        activations[:,offset:offset+window_size]=activations[:,offset:offset+window_size]+probabilities
        window_index += 1
